In [2]:
!pip install playwright
!playwright install
!brew install ffmpeg   # macOS
# or: sudo apt install ffmpeg


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/core/portable-ruby/blobs/sha256:1c98fa49eacc935640a6f8e10a2bf33f14cfc276804b71ddb658ea45ba99d167
######################################################################### 100.0%
==> Pouring portable-ruby-3.4.8.arm64_big_sur.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
aklomp-base64: Fast Base64 stream encoder/decoder in C99, with SIMD acceleration
ansible@12: Automate deployment, configuration, and upgrading
astra: Command-Line Interface for DataStax Astra
auto-editor: Efficient media analysis and rendering
aws-spiffe-workload-helper: Helper for providing AWS credential

In [ ]:
import os, asyncio
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 30,
    seconds: float = 5.0,
    width: int = 1280,
    height: int = 720,
    device_scale_factor: int = 1,
    wait_before_start_ms: int = 300,  # let fonts/layout settle
):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    # If it's a local file, Playwright wants file://
    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * seconds)
    frame_interval = 1.0 / fps

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")
        await page.wait_for_timeout(wait_before_start_ms)

        start = asyncio.get_event_loop().time()

        for i in range(total_frames):
            # Try to sample frames evenly in "wall time"
            target_t = start + i * frame_interval
            now = asyncio.get_event_loop().time()
            if target_t > now:
                await asyncio.sleep(target_t - now)

            frame_path = out / f"frame_{i:05d}.png"
            await page.screenshot(path=str(frame_path), full_page=False)

        await browser.close()

    return total_frames

# ✅ Run it in Jupyter with await:
# Example: local file "anim.html" for 4 seconds @ 30 fps
total = await html_to_frames("dark.html", out_dir="frames", fps=60, seconds=11, width=1920, height=1080,device_scale_factor=1.2)
total



660

In [15]:
import subprocess, shlex

def frames_to_mp4(frames_dir="frames", fps=30, out_mp4="out_dark.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -pix_fmt yuv420p -crf 18 -preset medium {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

mp4_path = frames_to_mp4("frames", fps=60, out_mp4="animation_dark.mp4")
mp4_path




ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

'animation_dark.mp4'

In [16]:
import os, asyncio
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 30,
    seconds: float = 5.0,
    width: int = 1280,
    height: int = 720,
    device_scale_factor: int = 1,
    wait_before_start_ms: int = 300,  # let fonts/layout settle
):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    # If it's a local file, Playwright wants file://
    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * seconds)
    frame_interval = 1.0 / fps

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")
        await page.wait_for_timeout(wait_before_start_ms)

        start = asyncio.get_event_loop().time()

        for i in range(total_frames):
            # Try to sample frames evenly in "wall time"
            target_t = start + i * frame_interval
            now = asyncio.get_event_loop().time()
            if target_t > now:
                await asyncio.sleep(target_t - now)

            frame_path = out / f"frame_{i:05d}.png"
            await page.screenshot(path=str(frame_path), full_page=False)

        await browser.close()

    return total_frames

# ✅ Run it in Jupyter with await:
# Example: local file "anim.html" for 4 seconds @ 30 fps
total = await html_to_frames("white.html", out_dir="frames", fps=40, seconds=14, width=1920, height=1080,device_scale_factor=1.2)
total

560

In [17]:
mp4_path = frames_to_mp4("frames_white", fps=40, out_mp4="animation_white.mp4")
mp4_path




ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

CalledProcessError: Command '['ffmpeg', '-y', '-framerate', '40', '-i', 'frames_white/frame_%05d.png', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-crf', '18', '-preset', 'medium', 'animation_white.mp4']' returned non-zero exit status 254.

In [18]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    seconds: float = 11.0,
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2, # 2.0 = HiDPI/Retina for razor sharp text
):
    out = Path(out_dir)
    if out.exists(): # Clear old frames to prevent mixing
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * seconds)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # --- DETERMINISTIC TIME CONTROL ---
        # This stops the browser's internal clock so we can step through frames
        await page.add_init_script("window.pauseAnimations = true;") 
        
        for i in range(total_frames):
            # Tell the browser exactly what time it should be
            current_time_ms = (i / fps) * 1000
            
            # This JS snippet forces CSS and Web Animations to the specific timestamp
            await page.evaluate(f"document.getAnimations().forEach(a => a.currentTime = {current_time_ms});")
            
            frame_path = out / f"frame_{i:05d}.png"
            await page.screenshot(path=str(frame_path), full_page=False, animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_dark.mp4"):
    # -crf 15: Very high quality (0 is lossless, 18 is standard high, 23 is default)
    # -preset slower: Spends more CPU time to ensure better compression/detail
    # -tune animation: Optimizes for the flat colors and sharp edges of HTML
    # -pix_fmt yuv420p: Best compatibility for players/web
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# Execution
total = await html_to_frames("dark.html", fps=60, seconds=11)
mp4_path = frames_to_mp4(fps=60)
print(f"Video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Video saved to: animation_dark.mp4


[out#0/mp4 @ 0x7a3031bc0] video:510KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.348751%
frame=  660 fps= 87 q=-1.0 Lsize=     516KiB time=00:00:10.96 bitrate= 385.8kbits/s speed=1.44x elapsed=0:00:07.60    
[libx264 @ 0x7a2c24a80] frame I:3     Avg QP: 6.95  size: 81271
[libx264 @ 0x7a2c24a80] frame P:111   Avg QP:10.58  size:  1260
[libx264 @ 0x7a2c24a80] frame B:546   Avg QP:13.75  size:   252
[libx264 @ 0x7a2c24a80] consecutive B-frames:  0.5%  0.0%  1.4%  0.0%  0.0% 98.2%
[libx264 @ 0x7a2c24a80] mb I  I16..4: 92.8%  4.5%  2.7%
[libx264 @ 0x7a2c24a80] mb P  I16..4:  0.0%  0.0%  0.0%  P16..4:  0.1%  0.0%  0.0%  0.0%  0.0%    skip:99.8%
[libx264 @ 0x7a2c24a80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.1%  0.0%  0.0%  direct: 0.0%  skip:99.9%  L0:57.7% L1:42.2% BI: 0.0%
[libx264 @ 0x7a2c24a80] 8x8 transform intra:4.5% inter:2.3%
[libx264 @ 0x7a2c24a80] direct mvs  spatial:76.6% temporal:23.4%
[libx264 @ 0x7a2c24a80] coded y,uvDC,uvAC intra: 2

In [19]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    seconds: float = 11.0,
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2, # 2.0 = HiDPI/Retina for razor sharp text
):
    out = Path(out_dir)
    if out.exists(): # Clear old frames to prevent mixing
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * seconds)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # --- DETERMINISTIC TIME CONTROL ---
        # This stops the browser's internal clock so we can step through frames
        await page.add_init_script("window.pauseAnimations = true;") 
        
        for i in range(total_frames):
            # Tell the browser exactly what time it should be
            current_time_ms = (i / fps) * 1000
            
            # This JS snippet forces CSS and Web Animations to the specific timestamp
            await page.evaluate(f"document.getAnimations().forEach(a => a.currentTime = {current_time_ms});")
            
            frame_path = out / f"frame_{i:05d}.png"
            await page.screenshot(path=str(frame_path), full_page=False, animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_white.mp4"):
    # -crf 15: Very high quality (0 is lossless, 18 is standard high, 23 is default)
    # -preset slower: Spends more CPU time to ensure better compression/detail
    # -tune animation: Optimizes for the flat colors and sharp edges of HTML
    # -pix_fmt yuv420p: Best compatibility for players/web
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# Execution
total = await html_to_frames("white.html", fps=60, seconds=11)
mp4_path = frames_to_mp4(fps=60)
print(f"Video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Video saved to: animation_white.mp4


[out#0/mp4 @ 0xa8900c900] video:494KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.390876%
frame=  660 fps= 88 q=-1.0 Lsize=     501KiB time=00:00:10.96 bitrate= 374.3kbits/s speed=1.46x elapsed=0:00:07.49    
[libx264 @ 0xa88c38a80] frame I:3     Avg QP: 8.35  size: 76611
[libx264 @ 0xa88c38a80] frame P:111   Avg QP:12.68  size:  1229
[libx264 @ 0xa88c38a80] frame B:546   Avg QP:13.75  size:   255
[libx264 @ 0xa88c38a80] consecutive B-frames:  0.5%  0.0%  1.4%  0.0%  0.0% 98.2%
[libx264 @ 0xa88c38a80] mb I  I16..4: 68.1% 29.3%  2.6%
[libx264 @ 0xa88c38a80] mb P  I16..4:  0.0%  0.0%  0.0%  P16..4:  0.1%  0.0%  0.0%  0.0%  0.0%    skip:99.8%
[libx264 @ 0xa88c38a80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.1%  0.0%  0.0%  direct: 0.0%  skip:99.9%  L0:56.4% L1:43.6% BI: 0.0%
[libx264 @ 0xa88c38a80] 8x8 transform intra:29.1% inter:1.9%
[libx264 @ 0xa88c38a80] direct mvs  spatial:78.2% temporal:21.8%
[libx264 @ 0xa88c38a80] coded y,uvDC,uvAC intra: 

In [20]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    target_seconds: float = 11.0, # The length of the final video
    original_seconds: float = 2.0, # The length of the actual CSS animation
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2,
):
    out = Path(out_dir)
    if out.exists():
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * target_seconds)
    
    # This ratio slows down the playback
    # If time_scale is 0.18, 1 second of video only moves 0.18 seconds of animation
    time_scale = original_seconds / target_seconds

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # Freeze animations initially
        await page.add_init_script("document.getAnimations().forEach(a => a.pause());") 
        
        for i in range(total_frames):
            # Calculate the "Slow Motion" timestamp
            # (Frame Index / FPS) gives us the video second
            # Multiplying by time_scale slows the animation down to fit the 11s
            video_time_ms = (i / fps) * 1000
            animation_time_ms = video_time_ms * time_scale
            
            # Inject the scaled time into the browser's animation engine
            await page.evaluate(f"""
                document.getAnimations().forEach(anim => {{
                    anim.currentTime = {animation_time_ms};
                }});
            """)
            
            frame_path = out / f"frame_{i:05d}.png"
            await page.screenshot(path=str(frame_path), animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_slow.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# --- RUN THE PROCESS ---
# We tell it the video should be 11 seconds, but the HTML animation is only 2 seconds long.
total = await html_to_frames("dark.html", fps=60, target_seconds=11.0, original_seconds=2.0)
mp4_path = frames_to_mp4(fps=60)
print(f"Slow-motion video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Slow-motion video saved to: animation_slow.mp4


[out#0/mp4 @ 0x8ad00c900] video:510KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.348586%
frame=  660 fps= 88 q=-1.0 Lsize=     517KiB time=00:00:10.96 bitrate= 385.8kbits/s speed=1.46x elapsed=0:00:07.49    
[libx264 @ 0x8acc60a80] frame I:3     Avg QP: 6.95  size: 81271
[libx264 @ 0x8acc60a80] frame P:111   Avg QP:10.53  size:  1260
[libx264 @ 0x8acc60a80] frame B:546   Avg QP:13.75  size:   252
[libx264 @ 0x8acc60a80] consecutive B-frames:  0.5%  0.0%  1.4%  0.0%  0.0% 98.2%
[libx264 @ 0x8acc60a80] mb I  I16..4: 92.8%  4.5%  2.7%
[libx264 @ 0x8acc60a80] mb P  I16..4:  0.0%  0.0%  0.0%  P16..4:  0.1%  0.0%  0.0%  0.0%  0.0%    skip:99.8%
[libx264 @ 0x8acc60a80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.1%  0.0%  0.0%  direct: 0.0%  skip:99.9%  L0:56.2% L1:43.7% BI: 0.1%
[libx264 @ 0x8acc60a80] 8x8 transform intra:4.6% inter:2.4%
[libx264 @ 0x8acc60a80] direct mvs  spatial:76.6% temporal:23.4%
[libx264 @ 0x8acc60a80] coded y,uvDC,uvAC intra: 2

In [21]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    target_seconds: float = 11.0, # The length of the final video
    original_seconds: float = 2.0, # The length of the actual CSS animation
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2,
):
    out = Path(out_dir)
    if out.exists():
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * target_seconds)
    
    # This ratio slows down the playback
    # If time_scale is 0.18, 1 second of video only moves 0.18 seconds of animation
    time_scale = original_seconds / target_seconds

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # Freeze animations initially
        await page.add_init_script("document.getAnimations().forEach(a => a.pause());") 
        
        for i in range(total_frames):
            # Calculate the "Slow Motion" timestamp
            # (Frame Index / FPS) gives us the video second
            # Multiplying by time_scale slows the animation down to fit the 11s
            video_time_ms = (i / fps) * 1000
            animation_time_ms = video_time_ms * time_scale
            
            # Inject the scaled time into the browser's animation engine
            await page.evaluate(f"""
                document.getAnimations().forEach(anim => {{
                    anim.currentTime = {animation_time_ms};
                }});
            """)
            
            frame_path = out / f"frame_{i:05d}.png"
            await page.screenshot(path=str(frame_path), animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_slow_white.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# --- RUN THE PROCESS ---
# We tell it the video should be 11 seconds, but the HTML animation is only 2 seconds long.
total = await html_to_frames("white.html", fps=60, target_seconds=11.0, original_seconds=2.0)
mp4_path = frames_to_mp4(fps=60)
print(f"Slow-motion video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Slow-motion video saved to: animation_slow_white.mp4


[out#0/mp4 @ 0xb8f409bc0] video:494KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.390810%
frame=  660 fps= 88 q=-1.0 Lsize=     501KiB time=00:00:10.96 bitrate= 374.3kbits/s speed=1.47x elapsed=0:00:07.46    
[libx264 @ 0xb8ec3ca80] frame I:3     Avg QP: 8.36  size: 76611
[libx264 @ 0xb8ec3ca80] frame P:111   Avg QP:12.50  size:  1229
[libx264 @ 0xb8ec3ca80] frame B:546   Avg QP:13.71  size:   255
[libx264 @ 0xb8ec3ca80] consecutive B-frames:  0.5%  0.0%  1.4%  0.0%  0.0% 98.2%
[libx264 @ 0xb8ec3ca80] mb I  I16..4: 68.1% 29.3%  2.6%
[libx264 @ 0xb8ec3ca80] mb P  I16..4:  0.0%  0.0%  0.0%  P16..4:  0.1%  0.0%  0.0%  0.0%  0.0%    skip:99.8%
[libx264 @ 0xb8ec3ca80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.1%  0.0%  0.0%  direct: 0.0%  skip:99.9%  L0:55.9% L1:44.1% BI: 0.0%
[libx264 @ 0xb8ec3ca80] 8x8 transform intra:29.1% inter:2.6%
[libx264 @ 0xb8ec3ca80] direct mvs  spatial:78.2% temporal:21.8%
[libx264 @ 0xb8ec3ca80] coded y,uvDC,uvAC intra: 

In [28]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    target_seconds: float = 11.0, 
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2,
):
    out = Path(out_dir)
    if out.exists():
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * target_seconds)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # --- THE FIX: STRETCH ANIMATION LIFESPAN ---
        # This script finds all animations and forces them to have an 11s duration
        # so they don't stop at the 2s mark.
        await page.evaluate(f"""
            const targetDuration = {target_seconds * 1000};
            document.getAnimations().forEach(anim => {{
                if (anim.effect) {{
                    anim.effect.updateTiming({{
                        duration: targetDuration,
                        fill: 'forwards' 
                    }});
                }}
                anim.pause();
            }});
        """)
        
        for i in range(total_frames):
            # Calculate time (linear progression from 0 to 11s)
            current_time_ms = (i / fps) * 1000
            
            # Update all animations to the current frame's timestamp
            await page.evaluate(f"""
                document.getAnimations().forEach(anim => {{
                    anim.currentTime = {current_time_ms};
                }});
            """)
            
            frame_path = out / f"frame_{i:05d}.png"
            # animations="disabled" prevents the browser from trying to play 
            # them naturally while we are taking the snapshot
            await page.screenshot(path=str(frame_path), animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_stretched.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# --- RUN ---
total = await html_to_frames("dark.html", fps=60, target_seconds=3.2)
mp4_path = frames_to_mp4(fps=60)
print(f"11-second high-quality video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

11-second high-quality video saved to: animation_stretched.mp4


[out#0/mp4 @ 0xb48c28d80] video:176KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.473470%
frame=  192 fps= 78 q=-1.0 Lsize=     179KiB time=00:00:03.16 bitrate= 463.1kbits/s speed=1.28x elapsed=0:00:02.47    
[libx264 @ 0xb4902ca80] frame I:1     Avg QP: 8.66  size: 18697
[libx264 @ 0xb4902ca80] frame P:32    Avg QP:12.97  size:  3655
[libx264 @ 0xb4902ca80] frame B:159   Avg QP:13.98  size:   279
[libx264 @ 0xb4902ca80] consecutive B-frames:  0.5%  0.0%  0.0%  0.0%  2.6% 96.9%
[libx264 @ 0xb4902ca80] mb I  I16..4: 99.4%  0.0%  0.6%
[libx264 @ 0xb4902ca80] mb P  I16..4:  0.0%  0.0%  0.1%  P16..4:  0.3%  0.0%  0.1%  0.0%  0.0%    skip:99.5%
[libx264 @ 0xb4902ca80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.2%  0.0%  0.0%  direct: 0.0%  skip:99.8%  L0:58.3% L1:41.6% BI: 0.1%
[libx264 @ 0xb4902ca80] 8x8 transform intra:0.1% inter:2.7%
[libx264 @ 0xb4902ca80] direct mvs  spatial:92.5% temporal:7.5%
[libx264 @ 0xb4902ca80] coded y,uvDC,uvAC intra: 2.

In [29]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    target_seconds: float = 11.0, 
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2,
):
    out = Path(out_dir)
    if out.exists():
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * target_seconds)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # --- THE FIX: STRETCH ANIMATION LIFESPAN ---
        # This script finds all animations and forces them to have an 11s duration
        # so they don't stop at the 2s mark.
        await page.evaluate(f"""
            const targetDuration = {target_seconds * 1000};
            document.getAnimations().forEach(anim => {{
                if (anim.effect) {{
                    anim.effect.updateTiming({{
                        duration: targetDuration,
                        fill: 'forwards' 
                    }});
                }}
                anim.pause();
            }});
        """)
        
        for i in range(total_frames):
            # Calculate time (linear progression from 0 to 11s)
            current_time_ms = (i / fps) * 1000
            
            # Update all animations to the current frame's timestamp
            await page.evaluate(f"""
                document.getAnimations().forEach(anim => {{
                    anim.currentTime = {current_time_ms};
                }});
            """)
            
            frame_path = out / f"frame_{i:05d}.png"
            # animations="disabled" prevents the browser from trying to play 
            # them naturally while we are taking the snapshot
            await page.screenshot(path=str(frame_path), animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_stretched_white.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# --- RUN ---
total = await html_to_frames("white.html", fps=60, target_seconds=3.2)
mp4_path = frames_to_mp4(fps=60)
print(f"11-second high-quality video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

11-second high-quality video saved to: animation_stretched_white.mp4


[out#0/mp4 @ 0xbd8c3c240] video:172KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.511221%
frame=  192 fps= 78 q=-1.0 Lsize=     175KiB time=00:00:03.16 bitrate= 451.7kbits/s speed=1.29x elapsed=0:00:02.45    
[libx264 @ 0xbd904ca80] frame I:1     Avg QP: 8.64  size: 18027
[libx264 @ 0xbd904ca80] frame P:32    Avg QP:13.09  size:  3513
[libx264 @ 0xbd904ca80] frame B:159   Avg QP:14.13  size:   283
[libx264 @ 0xbd904ca80] consecutive B-frames:  0.5%  0.0%  0.0%  0.0%  2.6% 96.9%
[libx264 @ 0xbd904ca80] mb I  I16..4: 99.4%  0.0%  0.6%
[libx264 @ 0xbd904ca80] mb P  I16..4:  0.0%  0.0%  0.1%  P16..4:  0.3%  0.1%  0.1%  0.0%  0.0%    skip:99.5%
[libx264 @ 0xbd904ca80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.2%  0.0%  0.0%  direct: 0.0%  skip:99.8%  L0:58.5% L1:41.5% BI: 0.0%
[libx264 @ 0xbd904ca80] 8x8 transform intra:0.2% inter:2.4%
[libx264 @ 0xbd904ca80] direct mvs  spatial:92.5% temporal:7.5%
[libx264 @ 0xbd904ca80] coded y,uvDC,uvAC intra: 2.

In [30]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    target_seconds: float = 11.0, 
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2,
):
    out = Path(out_dir)
    if out.exists():
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * target_seconds)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # --- THE FIX: STRETCH ANIMATION LIFESPAN ---
        # This script finds all animations and forces them to have an 11s duration
        # so they don't stop at the 2s mark.
        await page.evaluate(f"""
            const targetDuration = {target_seconds * 1000};
            document.getAnimations().forEach(anim => {{
                if (anim.effect) {{
                    anim.effect.updateTiming({{
                        duration: targetDuration,
                        fill: 'forwards' 
                    }});
                }}
                anim.pause();
            }});
        """)
        
        for i in range(total_frames):
            # Calculate time (linear progression from 0 to 11s)
            current_time_ms = (i / fps) * 1000
            
            # Update all animations to the current frame's timestamp
            await page.evaluate(f"""
                document.getAnimations().forEach(anim => {{
                    anim.currentTime = {current_time_ms};
                }});
            """)
            
            frame_path = out / f"frame_{i:05d}.png"
            # animations="disabled" prevents the browser from trying to play 
            # them naturally while we are taking the snapshot
            await page.screenshot(path=str(frame_path), animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_stretched_white_title.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# --- RUN ---
total = await html_to_frames("white_title.html", fps=60, target_seconds=3.2)
mp4_path = frames_to_mp4(fps=60)
print(f"11-second high-quality video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

11-second high-quality video saved to: animation_stretched_white_title.mp4


[out#0/mp4 @ 0x81f00c900] video:402KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.646589%
frame=  192 fps= 74 q=-1.0 Lsize=     405KiB time=00:00:03.16 bitrate=1046.8kbits/s speed=1.22x elapsed=0:00:02.59    
[libx264 @ 0x81e808a80] frame I:1     Avg QP: 6.94  size: 18029
[libx264 @ 0x81e808a80] frame P:32    Avg QP:16.74  size: 10595
[libx264 @ 0x81e808a80] frame B:159   Avg QP:15.56  size:   339
[libx264 @ 0x81e808a80] consecutive B-frames:  0.5%  0.0%  0.0%  0.0%  2.6% 96.9%
[libx264 @ 0x81e808a80] mb I  I16..4: 99.4%  0.0%  0.6%
[libx264 @ 0x81e808a80] mb P  I16..4:  0.0%  0.2%  0.2%  P16..4:  1.5%  0.2%  0.3%  0.1%  0.0%    skip:97.6%
[libx264 @ 0x81e808a80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.5%  0.0%  0.0%  direct: 0.0%  skip:99.5%  L0:57.3% L1:42.7% BI: 0.0%
[libx264 @ 0x81e808a80] 8x8 transform intra:4.8% inter:1.8%
[libx264 @ 0x81e808a80] direct mvs  spatial:92.5% temporal:7.5%
[libx264 @ 0x81e808a80] coded y,uvDC,uvAC intra: 3.

In [31]:
import os, asyncio, subprocess, shlex
from pathlib import Path
from playwright.async_api import async_playwright

async def html_to_frames(
    html_path_or_url: str,
    out_dir: str = "frames",
    fps: int = 60,
    target_seconds: float = 11.0, 
    width: int = 1920,
    height: int = 1080,
    device_scale_factor: int = 2,
):
    out = Path(out_dir)
    if out.exists():
        for f in out.glob("*.png"): f.unlink()
    out.mkdir(parents=True, exist_ok=True)

    target = html_path_or_url
    if Path(html_path_or_url).exists():
        target = Path(html_path_or_url).resolve().as_uri()

    total_frames = int(fps * target_seconds)

    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(
            viewport={"width": width, "height": height},
            device_scale_factor=device_scale_factor,
        )
        page = await context.new_page()
        await page.goto(target, wait_until="networkidle")

        # --- THE FIX: STRETCH ANIMATION LIFESPAN ---
        # This script finds all animations and forces them to have an 11s duration
        # so they don't stop at the 2s mark.
        await page.evaluate(f"""
            const targetDuration = {target_seconds * 1000};
            document.getAnimations().forEach(anim => {{
                if (anim.effect) {{
                    anim.effect.updateTiming({{
                        duration: targetDuration,
                        fill: 'forwards' 
                    }});
                }}
                anim.pause();
            }});
        """)
        
        for i in range(total_frames):
            # Calculate time (linear progression from 0 to 11s)
            current_time_ms = (i / fps) * 1000
            
            # Update all animations to the current frame's timestamp
            await page.evaluate(f"""
                document.getAnimations().forEach(anim => {{
                    anim.currentTime = {current_time_ms};
                }});
            """)
            
            frame_path = out / f"frame_{i:05d}.png"
            # animations="disabled" prevents the browser from trying to play 
            # them naturally while we are taking the snapshot
            await page.screenshot(path=str(frame_path), animations="disabled")

        await browser.close()
    return total_frames

def frames_to_mp4(frames_dir="frames", fps=60, out_mp4="animation_stretched_title.mp4"):
    cmd = f"""
    ffmpeg -y -framerate {fps} -i {frames_dir}/frame_%05d.png
           -c:v libx264 -crf 15 -preset slower -tune animation 
           -pix_fmt yuv420p {out_mp4}
    """
    subprocess.run(shlex.split(" ".join(cmd.split())), check=True)
    return out_mp4

# --- RUN ---
total = await html_to_frames("dark_title.html", fps=60, target_seconds=3.2)
mp4_path = frames_to_mp4(fps=60)
print(f"11-second high-quality video saved to: {mp4_path}")

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

11-second high-quality video saved to: animation_stretched_title.mp4


[out#0/mp4 @ 0xab1400900] video:400KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.649821%
frame=  192 fps= 73 q=-1.0 Lsize=     403KiB time=00:00:03.16 bitrate=1041.6kbits/s speed= 1.2x elapsed=0:00:02.64    
[libx264 @ 0xab0818a80] frame I:1     Avg QP: 7.14  size: 18701
[libx264 @ 0xab0818a80] frame P:32    Avg QP:17.54  size: 10540
[libx264 @ 0xab0818a80] frame B:159   Avg QP:14.88  size:   333
[libx264 @ 0xab0818a80] consecutive B-frames:  0.5%  0.0%  0.0%  0.0%  2.6% 96.9%
[libx264 @ 0xab0818a80] mb I  I16..4: 99.4%  0.0%  0.6%
[libx264 @ 0xab0818a80] mb P  I16..4:  0.0%  0.2%  0.2%  P16..4:  1.2%  0.2%  0.3%  0.1%  0.0%    skip:97.9%
[libx264 @ 0xab0818a80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.5%  0.0%  0.0%  direct: 0.0%  skip:99.5%  L0:57.3% L1:42.7% BI: 0.0%
[libx264 @ 0xab0818a80] 8x8 transform intra:4.5% inter:1.5%
[libx264 @ 0xab0818a80] direct mvs  spatial:92.5% temporal:7.5%
[libx264 @ 0xab0818a80] coded y,uvDC,uvAC intra: 3.